In [ ]:
#!pip install sharepy
import sharepy 
import os
#System variables
MAIL_NUB = os.environ.get('MAIL_NUB')
PASS_MAIL_NUB = os.environ.get('PASS_MAIL_NUB')
s = sharepy.connect("nubsieteocho.sharepoint.com", MAIL_NUB, PASS_MAIL_NUB)

In [ ]:
import smtplib
import pandas as pd
from datetime import datetime, timedelta
from pandas import Timestamp
import excel_formating

from email import encoders
from os.path import basename
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from string import printable

In [ ]:
#import data with the users mails
p = s.getfile("https://nubsieteocho.sharepoint.com/sites/ODAOPS/Shared%20Documents/General/CAB%20Trackers/Coverage%20Advisor%20Tracker%20OFICIAL.xlsx"\
    ,filename = 'Coverage Advisor Tracker OFICIAL.xlsx')
ca = pd.read_excel ('Coverage Advisor Tracker OFICIAL.xlsx')
ca.dropna(axis=0, subset=['Account'], inplace=True)
ca.drop(list(ca[ca['Account']==0].index), axis=0, inplace=True)
ca.dropna(axis=0, subset=['Request ID'], inplace=True)

In [ ]:
m = s.getfile("https://nubsieteocho.sharepoint.com/sites/ODAOPS/Shared%20Documents/General/coverage_mails.xlsx"\
    ,filename = 'coverage_mails.xlsx')
mail = pd.read_excel('coverage_mails.xlsx')

In [ ]:
#SMTP Mail variables
today = datetime.strftime(datetime.today(), '%m-%d-%Y')
myMail = 'testNub78@outlook.com'
password = 'EasyP4ssword78'
subjectEmail = 'Coverage error report'

msg1 = """<h1>Hola</h1>
        <h3>Por favor revisa el archivo adjunto para encontrar el detalle de los errores</h3>
        <h3>Asegurate de que tus registros esten corregidos y al d&iacute;a</h3>"""

In [ ]:


def format_date_column(serie):
    format_date = []
    for index, value in serie.iteritems():
        if type(value) != datetime and type(value) != Timestamp:  
            if pd.isnull(value) or value != value:
                #Fechas vacias
                value = datetime.strptime('2030-2-2 00:00:00', '%Y-%m-%d %H:%M:%S')
                format_date.append(value)
            else: 
                #Error de formato
                value = datetime.strptime('2050-2-2 00:00:00', '%Y-%m-%d %H:%M:%S')
                format_date.append(value)
        else:
            #Fechas normales
            value = datetime.strptime(str(value), '%Y-%m-%d %H:%M:%S')
            format_date.append(value)
    
    return format_date
    


In [ ]:
# Request Type

request_type = ca[['Request ID', 'Request Type']].copy()

for i, row in request_type.iterrows():
    if pd.isnull(row['Request Type']) == True:
        request_type['Request Type'].loc[i] = 'Request Type Empty'
    else:
        request_type['Request Type'].loc[i] = None

request_type.dropna(axis=0, subset=['Request Type'], inplace=True)


In [ ]:
# Join Call

join_call = ca[['Request ID', 'Join Call']].copy()

for i, row in join_call.iterrows():
    if pd.isnull(row['Join Call']) == True:
        join_call['Join Call'].loc[i] = 'Join Call Empty'
    else:
        join_call['Join Call'].loc[i] = None

join_call.dropna(axis=0, subset=['Join Call'], inplace=True)


In [ ]:
# Upcoming Renewal
upcoming_renewal = ca[['Request ID', 'Upcoming Renewal']].copy()

for index, row in upcoming_renewal.iterrows():
    if pd.isna(row['Upcoming Renewal']) == True:
        upcoming_renewal['Upcoming Renewal'].loc[index] = 'Empty Value'
    elif len(row['Upcoming Renewal']) > 6:
        upcoming_renewal['Upcoming Renewal'].loc[index] = 'Value with space'
    else:    
        upcoming_renewal['Upcoming Renewal'].loc[index] = None

upcoming_renewal.dropna(axis=0, subset=['Upcoming Renewal'], inplace=True)


In [ ]:
# Date Received

date_received = ca[['Request ID', 'Date received', 'PAR Requested']].copy()
date_received['Date received'] = format_date_column(date_received['Date received'])
date_received['PAR Requested'] = format_date_column(date_received['PAR Requested'])

for index, row in date_received.iterrows():
    if row['Date received'] == Timestamp('2030-2-2 00:00:00'):
        date_received['Date received'].loc[index] = 'Empty Date'
    elif row['Date received'] == Timestamp('2050-2-2 00:00:00'):
        date_received['Date received'].loc[index] = 'Format Error'
    elif row['Date received'] < Timestamp('2021-10-15 00:00:00'):
        date_received['Date received'].loc[index] = '15/oct error'
    elif row['Date received'] > row['PAR Requested']:
        if row['PAR Requested'] != (Timestamp('2030-2-2 00:00:00') or Timestamp('2050-2-2 00:00:00')):
            date_received['Date received'].loc[index] = 'Date greater than PAR requested'
        else:
            date_received['Date received'].loc[index] = None
    else:
        date_received['Date received'].loc[index] = None

date_received.dropna(axis=0, subset=['Date received'], inplace=True)
date_received.drop(axis=1, columns=['PAR Requested'], inplace=True)

In [ ]:
date_received

In [ ]:
# Coverage Advisor

cov_adv = ca[['Request ID', 'Coverage Advisor']].copy()

for index, row in cov_adv.iterrows():
    if pd.isnull(row['Coverage Advisor']) == True:
        cov_adv['Coverage Advisor'].loc[index] = 'Empty Name'
    elif set(row['Coverage Advisor']).difference(printable):
        cov_adv['Coverage Advisor'].loc[index] = 'Special Caracter in Name'
    else:
        cov_adv['Coverage Advisor'].loc[index] = None

cov_adv.dropna(axis=0, subset=['Coverage Advisor'], inplace=True)

In [ ]:
# PAR Requested

PAR_req = ca[['Request ID', 'Date received', 'PAR Requested']].copy()
PAR_req['Date received'] = format_date_column(PAR_req['Date received'])
PAR_req['PAR Requested'] = format_date_column(PAR_req['PAR Requested'])

for index, row in PAR_req.iterrows():
    if row['PAR Requested'] == Timestamp('2030-2-2 00:00:00'):
        PAR_req['PAR Requested'].loc[index] = 'Empty Date'
    elif row['PAR Requested'] == Timestamp('2050-2-2 00:00:00'):
        PAR_req['PAR Requested'].loc[index] = 'Format Error'
    elif row['Date received'] > row['PAR Requested']:
        if row['Date received'] != (Timestamp('2030-2-2 00:00:00') or Timestamp('2050-2-2 00:00:00')):
            PAR_req['PAR Requested'].loc[index] = 'Date Received greater than PAR requested'
        else:
            PAR_req['PAR Requested'].loc[index] = None
    else:
        PAR_req['PAR Requested'].loc[index] = None

PAR_req.dropna(axis=0, subset=['PAR Requested'], inplace=True)
PAR_req.drop(axis=1, columns=['Date received'], inplace=True)

In [ ]:
# Date 1st contact
d_f_contact = ca[['Request ID', 'Date received', 'Date First Contact']].copy()
d_f_contact['Date received'] = format_date_column(d_f_contact['Date received'])
d_f_contact['Date First Contact'] = format_date_column(d_f_contact['Date First Contact'])

for index, row in d_f_contact.iterrows():
    if row['Date First Contact'] == Timestamp('2030-2-2 00:00:00'):
        d_f_contact['Date First Contact'].loc[index] = None
    elif row['Date First Contact'] == Timestamp('2050-2-2 00:00:00'):
        d_f_contact['Date First Contact'].loc[index] = 'Format Error'
    elif row['Date received'] > row['Date First Contact']:
        if row['Date received'] != (Timestamp('2030-2-2 00:00:00') or Timestamp('2050-2-2 00:00:00')):
            d_f_contact['Date First Contact'].loc[index] = 'Date Received greater than First Contact'
        else:
            d_f_contact['Date First Contact'].loc[index] = None
    else:
        d_f_contact['Date First Contact'].loc[index] = None

d_f_contact.dropna(axis=0, subset=['Date First Contact'], inplace=True)
d_f_contact.drop(axis=1, columns=['Date received'], inplace=True)

In [ ]:
# PAR Received

PAR_recev = ca[['Request ID', 'PAR Requested', 'PAR Received']].copy()
PAR_recev['PAR Requested'] = format_date_column(PAR_recev['PAR Requested'])
PAR_recev['PAR Received'] = format_date_column(PAR_recev['PAR Received'])

for index, row in PAR_recev.iterrows():
    if row['PAR Received'] == Timestamp('2030-2-2 00:00:00'):
        PAR_recev['PAR Received'].loc[index] = None
    elif row['PAR Received'] == Timestamp('2050-2-2 00:00:00'):
        PAR_recev['PAR Received'].loc[index] = 'Format Error'
    elif row['PAR Requested'] > row['PAR Received']:
        if row['PAR Requested'] != (Timestamp('2030-2-2 00:00:00') or Timestamp('2050-2-2 00:00:00')):
            PAR_recev['PAR Received'].loc[index] = 'PAR Requested greater than PAR received'
        else:
            PAR_recev['PAR Received'].loc[index] = None
    else:
        PAR_recev['PAR Received'].loc[index] = None

PAR_recev.dropna(axis=0, subset=['PAR Received'], inplace=True)
PAR_recev.drop(axis=1, columns=['PAR Requested'], inplace=True)

In [ ]:
# Date Analysis

dat_nalys = ca[['Request ID', 'PAR Received', 'Date Analysis']].copy()
dat_nalys['Date Analysis'] = format_date_column(dat_nalys['Date Analysis'])
dat_nalys['PAR Received'] = format_date_column(dat_nalys['PAR Received'])

for index, row in dat_nalys.iterrows():
    if row['Date Analysis'] == Timestamp('2030-2-2 00:00:00'):
        dat_nalys['Date Analysis'].loc[index] = None
    elif row['Date Analysis'] == Timestamp('2050-2-2 00:00:00'):
        dat_nalys['Date Analysis'].loc[index] = 'Format Error'
    elif row['PAR Received'] > row['Date Analysis']:
        if row['PAR Received'] != (Timestamp('2030-2-2 00:00:00') or Timestamp('2050-2-2 00:00:00')):
            dat_nalys['Date Analysis'].loc[index] = 'PAR received greater than Date Analysis'
        else:
            dat_nalys['Date Analysis'].loc[index] = None
    else:
        dat_nalys['Date Analysis'].loc[index] = None

dat_nalys.dropna(axis=0, subset=['Date Analysis'], inplace=True)
dat_nalys.drop(axis=1, columns=['PAR Received'], inplace=True)

In [ ]:
# Pre Readout Effort

pre_reffor = ca[['Request ID', 'Pre-Readout Effort (min)', 'Account Stage']].copy()

for j, row in pre_reffor.iterrows():
    if pd.isna(row['Pre-Readout Effort (min)']) == True:
        if pd.isna(row['Account Stage']) == False:
            if row['Account Stage'][0] == 'F':
                pre_reffor['Pre-Readout Effort (min)'].loc[j] = 'empty value'
    elif row['Pre-Readout Effort (min)'] > 180:
        pre_reffor['Pre-Readout Effort (min)'].loc[j] = 'value above 180'
    else:
        pre_reffor['Pre-Readout Effort (min)'].loc[j] = None

pre_reffor.dropna(axis=0, subset=['Pre-Readout Effort (min)'], inplace=True)
pre_reffor.drop(axis=1, columns=['Account Stage'], inplace=True)

In [ ]:
# Date Inite Sent

date_init_s = ca[['Request ID', 'Date First Contact', 'Date Invite Sent']].copy()
date_init_s['Date Invite Sent'] = format_date_column(date_init_s['Date Invite Sent'])
date_init_s['Date First Contact'] = format_date_column(date_init_s['Date First Contact'])

for index, row in date_init_s.iterrows():
    if row['Date Invite Sent'] == Timestamp('2030-2-2 00:00:00'):
        date_init_s['Date Invite Sent'].loc[index] = None
    elif row['Date Invite Sent'] == Timestamp('2050-2-2 00:00:00'):
        date_init_s['Date Invite Sent'].loc[index] = 'Format Error'
    elif row['Date First Contact'] > row['Date Invite Sent']:
        if row['Date First Contact'] != (Timestamp('2030-2-2 00:00:00') or Timestamp('2050-2-2 00:00:00')):
            date_init_s['Date Invite Sent'].loc[index] = 'Date First Contact greater than Date Invite Sent'
        else:
            date_init_s['Date Invite Sent'].loc[index] = None
    else:
        date_init_s['Date Invite Sent'].loc[index] = None

date_init_s.dropna(axis=0, subset=['Date Invite Sent'], inplace=True)
date_init_s.drop(axis=1, columns=['Date First Contact'], inplace=True)

In [ ]:
# Date Schedule

date_sche = ca[['Request ID', 'Date Invite Sent', 'Date Scheduled', 'Date Readout Done']].copy()
date_sche['Date Invite Sent'] = format_date_column(date_sche['Date Invite Sent'])
date_sche['Date Scheduled'] = format_date_column(date_sche['Date Scheduled'])
date_sche['Date Readout Done'] = format_date_column(date_sche['Date Readout Done'])

for index, row in date_sche.iterrows():
    if row['Date Scheduled'] == Timestamp('2030-2-2 00:00:00'):
        date_sche['Date Scheduled'].loc[index] = None
    elif row['Date Scheduled'] == Timestamp('2050-2-2 00:00:00'):
        date_sche['Date Scheduled'].loc[index] = 'Format Error'
    elif row['Date Scheduled'] != row['Date Readout Done']:
        if row['Date Readout Done'] != (Timestamp('2030-2-2 00:00:00') or Timestamp('2050-2-2 00:00:00')):
            date_sche['Date Scheduled'].loc[index] = 'Scheduled and Readout Done are different'
        else:
            date_sche['Date Scheduled'].loc[index] = None
    elif row['Date Invite Sent'] > row['Date Scheduled']:
        if row['Date Invite Sent'] != (Timestamp('2030-2-2 00:00:00') or Timestamp('2050-2-2 00:00:00')):
            date_sche['Date Scheduled'].loc[index] = 'Invite Sent greater than Scheduled'
        else:
            date_sche['Date Scheduled'].loc[index] = None
    else:
        date_sche['Date Scheduled'].loc[index] = None

date_sche.dropna(axis=0, subset=['Date Scheduled'], inplace=True)
date_sche.drop(axis=1, columns=['Date Invite Sent'], inplace=True)
date_sche.drop(axis=1, columns=['Date Readout Done'], inplace=True)

In [ ]:
# Date Readout Done

date_red_don = ca[['Request ID', 'Date Scheduled', 'Date Readout Done', 'Status']].copy()
date_red_don['Date Scheduled'] = format_date_column(date_red_don['Date Scheduled'])
date_red_don['Date Readout Done'] = format_date_column(date_red_don['Date Readout Done'])

for index, row in date_red_don.iterrows():
    if row['Date Readout Done'] == Timestamp('2030-2-2 00:00:00'):
        if pd.isnull(row['Status']) == False:
            if row['Status'][0] != 'F':
                date_red_don['Date Readout Done'].loc[index] = 'Value mustnt be empty'
            else:
                date_red_don['Date Readout Done'].loc[index] = None
        else:
            date_red_don['Date Readout Done'].loc[index] = None
    elif row['Date Scheduled'] == Timestamp('2050-2-2 00:00:00'):
        date_red_don['Date Readout Done'].loc[index] = 'Format Error'
    elif row['Date Scheduled'] != row['Date Readout Done']:
        if row['Date Scheduled'] != (Timestamp('2030-2-2 00:00:00') or Timestamp('2050-2-2 00:00:00')):
            date_red_don['Date Readout Done'].loc[index] = 'Scheduled and Readout Done are different'
        else:
            date_red_don['Date Readout Done'].loc[index] = None
    else:
        date_red_don['Date Readout Done'].loc[index] = None

date_red_don.dropna(axis=0, subset=['Date Readout Done'], inplace=True)
date_red_don.drop(axis=1, columns=['Date Scheduled'], inplace=True)
date_red_don.drop(axis=1, columns=['Status'], inplace=True)

In [ ]:
# Readout Duration (min)

read_min = ca[['Request ID', 'Date Readout Done', 'Readout Duration (min)']].copy()

for j, row in read_min.iterrows():
    if pd.isnull(row['Date Readout Done']) == False and pd.isnull(row['Readout Duration (min)']) == True:
        read_min['Readout Duration (min)'].loc[j] = 'Value mustnt be empty'
    elif row['Readout Duration (min)'] > 180:
        read_min['Readout Duration (min)'].loc[j] = 'Value to high'
    else:
        read_min['Readout Duration (min)'].loc[j] = None

read_min.dropna(axis=0, subset=['Readout Duration (min)'], inplace=True)
read_min.drop(axis=1, columns=['Date Readout Done'], inplace=True)

In [ ]:
# Status

c_status = ca[['Request ID', 'Account Stage', 'Status']].copy()

for j, row in c_status.iterrows():
    if pd.isnull(row['Account Stage']) == False and pd.isnull(row['Status']) == True:
        c_status['Status'].loc[j] = 'must fill this cell'
    else:
        c_status['Status'].loc[j] = None

c_status.dropna(axis=0, subset=['Status'], inplace=True)
c_status.drop(axis=1, columns=['Account Stage'], inplace=True)

In [ ]:
# Delivery Type

del_type = ca[['Request ID', 'Date Readout Done', 'Delivery Type']].copy()

for id, row in del_type.iterrows():
    if pd.isnull(row['Delivery Type']) == True:
        del_type['Delivery Type'].loc[id] = 'Empty value'
    elif pd.isna(row['Date Readout Done']) == True and row['Delivery Type'] != 'Not presented':
        del_type['Delivery Type'].loc[id] = 'should be: Not Presented'
    else:
        del_type['Delivery Type'].loc[id] = None

del_type.dropna(axis=0, subset=['Delivery Type'], inplace=True)
del_type.drop(axis=1, columns=['Date Readout Done'], inplace=True)    

In [ ]:
# Reason Declined

reason_decl = ca[['Request ID', 'Status', 'Reason Declined']].copy()

for index, row in reason_decl.iterrows():
    if pd.isnull(row['Status']) == True:
        reason_decl['Reason Declined'].loc[index] = None
    elif row['Status'][2] == 'D' and pd.isnull(row['Reason Declined']) == True:
        reason_decl['Reason Declined'].loc[index] = 'Empty Value'
    else:
        reason_decl['Reason Declined'].loc[index] = None

reason_decl.dropna(axis=0, subset=['Reason Declined'], inplace=True)
reason_decl.drop(axis=1, columns=['Status'], inplace=True) 


In [ ]:
# Reason Declined Lvl 2

reason_decl2 = ca[['Request ID', 'Reason Declined', 'Reason Declined Lvl 2']].copy()

for index, row in reason_decl2.iterrows():
    if pd.isnull(row['Reason Declined']) == False and pd.isnull(row['Reason Declined Lvl 2']) == True:
        reason_decl2['Reason Declined Lvl 2'].loc[index] = 'Empty Value'
    else:
        reason_decl2['Reason Declined Lvl 2'].loc[index] = None

reason_decl2.dropna(axis=0, subset=['Reason Declined Lvl 2'], inplace=True)
reason_decl2.drop(axis=1, columns=['Reason Declined'], inplace=True) 

In [ ]:
# Reason On Hold and Days on Hold

reas_hold = ca[['Request ID', 'Reason On Hold', 'Days on Hold']].copy()

for index, row in reas_hold.iterrows():
    if pd.isna(row['Reason On Hold']) == True and pd.isna(row['Days on Hold']) == False:
        reas_hold['Reason On Hold'].loc[index] = 'Empty Value'
        reas_hold['Days on Hold'].loc[index] = None
    elif pd.isna(row['Reason On Hold']) == False and pd.isna(row['Days on Hold']) == True:
        reas_hold['Days on Hold'].loc[index] = 'Empty Value'
        reas_hold['Reason On Hold'].loc[index] = None
    else:
        reas_hold['Reason On Hold'].loc[index] = None
        reas_hold['Days on Hold'].loc[index] = None

reas_hold.dropna(axis=0, how='all', subset=['Reason On Hold', 'Days on Hold'], inplace=True) 

In [102]:
# Miss Attach Opportunity // Expired Opportunity // LDoS Opportunity

oppty= ca[['Request ID', 'Account Stage','Miss Attach Opportunity', 'Expired Opportunity', 'LDoS Opportunity']].copy()

for p, row in oppty.iterrows():
    if pd.isna(row['Account Stage']) == False:
        if pd.isna(row['Miss Attach Opportunity']) == True:
            oppty['Miss Attach Opportunity'].loc[p] == 'Empty oppotunity'
        if pd.isna(row['Expired Opportunity']) == True:
            oppty['Expired Opportunity'].loc[p] == 'Empty oppotunity'
        if pd.isna(row['LDoS Opportunity']) == True:
            oppty['LDoS Opportunity'].loc[p] == 'Empty oppotunity'


KeyError: 'Miss Attact Opportunity'

In [100]:
oppty

,Request ID,Account Stage,Miss Attach Opportunity,Expired Opportunity,LDoS Opportunity
0,103120.0,F_Engagement_Outcome,68264.0,187731,100350
1,102639.0,F_Engagement_Outcome,13000.0,58775,96000
2,102732.0,F_Engagement_Outcome,66000.0,61000,5684
3,103048.0,F_Engagement_Outcome,3458.0,2292,66205
4,103106.0,F_Engagement_Outcome,6800.0,62861,61473
...,...,...,...,...,...
1564,115650.0,NaN,NaN,NaN,NaN
1565,116119.0,NaN,NaN,NaN,NaN
1566,111891.0,NaN,NaN,NaN,NaN
1567,115641.0,NaN,NaN,NaN,NaN


In [ ]:
# Merging new tables
new_table = request_type.merge(join_call, how='outer', left_on='Request ID', right_on='Request ID').merge(
            upcoming_renewal, how='outer', left_on='Request ID', right_on='Request ID').merge(
            date_received, how='outer', left_on='Request ID', right_on='Request ID').merge(
            cov_adv, how='outer', left_on='Request ID', right_on='Request ID').merge(
            PAR_req, how='outer', left_on='Request ID', right_on='Request ID').merge(
            d_f_contact, how='outer', left_on='Request ID', right_on='Request ID').merge(
            PAR_recev, how='outer', left_on='Request ID', right_on='Request ID').merge(
            dat_nalys, how='outer', left_on='Request ID', right_on='Request ID').merge(
            pre_reffor, how='outer', left_on='Request ID', right_on='Request ID').merge(
            date_init_s, how='outer', left_on='Request ID', right_on='Request ID').merge(
            date_sche, how='outer', left_on='Request ID', right_on='Request ID').merge(
            date_red_don, how='outer', left_on='Request ID', right_on='Request ID').merge(
            read_min, how='outer', left_on='Request ID', right_on='Request ID').merge(
            c_status, how='outer', left_on='Request ID', right_on='Request ID').merge(
            del_type, how='outer', left_on='Request ID', right_on='Request ID').merge(
            reason_decl, how='outer', left_on='Request ID', right_on='Request ID').merge(
            reason_decl2, how='outer', left_on='Request ID', right_on='Request ID').merge(
            reas_hold, how='outer', left_on='Request ID', right_on='Request ID')


In [ ]:
new_table.to_csv('t.csv')

### Celdas vacías

In [ ]:
#Recorrido de celdas buscando valores vacios en columnas G H I J  AF/AG/AH AM AN AO
Empty  = ca[
    (ca['Request Type'].isna() == True) | (ca['Join Call'].isna() == True)
    | (ca['Sav ID'].isna() == True) | (ca['Upcoming Renewal'].isna() == True) | (ca['CA Name'].isna() == True) | (ca['Expired Opportunity'].isna() == True)
    | (ca['LDoS Opportunity'].isna() == True) | (ca['Next steps'].isna() == True) | (ca['Link PAR IB Rep'].isna() == True)]

Empty_Errors = Empty.copy()

In [ ]:
Empty_Errors = Empty_Errors[['Request ID','Request Type','Join Call','Sav ID', 'Upcoming Renewal','CA Name', 'Expired Opportunity','LDoS Opportunity', 'Next steps','Link PAR IB Rep']]
Empty_Errors = Empty_Errors.fillna('Empty')
Empty_Errors = Empty_Errors.reset_index(drop=True)

In [ ]:
for index, row in Empty_Errors.iterrows():
    for loc, sheet in row.items():
        if sheet != 'Empty':
            if loc != 'Request ID':
                Empty_Errors[loc].loc[index] = None

In [ ]:
Empty_Errors.rename(columns={'Request Type': 'Empty Request Type', 'Join Call': 'Empty Join Call',
                    'Sav ID': 'Empty Sav ID', 'Upcoming Renewal': 'Empty Upcoming Renewal',
                    'CA Name': 'Empty CA Name', 'Expired Opportunity': 'Empty Expired Opportunity',
                    'LDoS Opportunity': 'Empty LDoS Opportunity', 'Next steps':'Empty Next steps', 'Link PAR IB Rep':'Empty Link PAR IB Rep'}, inplace=True)

### Compare column R-W

In [ ]:
ca2 = ca[['Request ID','Coverage Advisor','PAR Requested','Date First Contact','PAR Received','Date Analysis','Date Invite Sent','Date Scheduled']]

In [ ]:
equality_relation = ca2.copy()

In [ ]:
equality_relation['PAR Requested'] = format_date_column(equality_relation['PAR Requested'])
equality_relation['Date First Contact'] = format_date_column(equality_relation['Date First Contact'])
equality_relation['PAR Received'] = format_date_column(equality_relation['PAR Received'])
equality_relation['Date Analysis'] = format_date_column(equality_relation['Date Analysis'])
equality_relation['Date Invite Sent'] = format_date_column(equality_relation['Date Analysis'])
equality_relation['Date Scheduled'] = format_date_column(equality_relation['Date Scheduled'])

In [ ]:
equality_relation.insert(equality_relation.shape[1], 'PAR Requested_greater_Date First Contact', None)
equality_relation.insert(equality_relation.shape[1], 'Date First Contact_greater_PAR Received', None)
equality_relation.insert(equality_relation.shape[1], 'PAR Received_greater_Date Analysis', None)
equality_relation.insert(equality_relation.shape[1], 'Date Analysis_greater_Date Invite Sent', None)
equality_relation.insert(equality_relation.shape[1], 'Date Invite Sent_greater_Date Scheduled', None)

In [ ]:
for i, value in equality_relation.iterrows():
    if value['PAR Requested'] > value['Date First Contact']:
        if value['PAR Requested'] == Timestamp('2030-2-2 00:00:00') or value['PAR Requested'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation['PAR Requested_greater_Date First Contact'].loc[i] = None
        else:
            equality_relation['PAR Requested_greater_Date First Contact'].loc[i] = 'X'

    if value['Date First Contact'] > value['PAR Received']:
        if value['Date First Contact'] == Timestamp('2030-2-2 00:00:00') or value['Date First Contact'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation['Date First Contact_greater_PAR Received'].loc[i] = None
        else:
            equality_relation['Date First Contact_greater_PAR Received'].loc[i] = 'X'  

    if value['PAR Received'] > value['Date Analysis']:
        if value['PAR Received'] == Timestamp('2030-2-2 00:00:00') or value['PAR Received'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation['PAR Received_greater_Date Analysis'].loc[i] = None  
        else:
            equality_relation['PAR Received_greater_Date Analysis'].loc[i] = 'X'

    if value['Date Analysis'] > value['Date Invite Sent']:
        if value['Date Analysis'] == Timestamp('2030-2-2 00:00:00') or value['Date Analysis'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation['Date Analysis_greater_Date Invite Sent'].loc[i] = None
        else:
            equality_relation['Date Analysis_greater_Date Invite Sent'].loc[i] = 'X'        

    if value['Date Invite Sent'] > value['Date Scheduled']:
        if value['Date Invite Sent'] == Timestamp('2030-2-2 00:00:00') or value['Date Invite Sent'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation['Date Invite Sent_greater_Date Scheduled'].loc[i] = None
        else: 
            equality_relation['Date Invite Sent_greater_Date Scheduled'].loc[i] = 'X' 

In [ ]:
for i, val in equality_relation.iterrows():

    if val['PAR Requested'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['PAR Requested'].loc[i] = 'Format Error'
    else:
        equality_relation['PAR Requested'].loc[i] = None

    if val['Date First Contact'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['Date First Contact'].loc[i] = 'Format Error'
    else:
        equality_relation['Date First Contact'].loc[i] = None
    
    if val['PAR Received'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['PAR Received'].loc[i] = 'Format Error'
    else:
        equality_relation['PAR Received'].loc[i] = None

    if val['Date Analysis'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['Date Analysis'].loc[i] = 'Format Error'
    else:
        equality_relation['Date Analysis'].loc[i] = None
    
    if val['Date Invite Sent'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['Date Invite Sent'].loc[i] = 'Format Error'
    else:
        equality_relation['Date Invite Sent'].loc[i] = None
    
    if val['Date Scheduled'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['Date Scheduled'].loc[i] = 'Format Error'
    else:
        equality_relation['Date Scheduled'].loc[i] = None



### Familiar Relation

In [ ]:

ca3 = ca[['Request ID','Account Stage','Status','Reason Declined','Pre-Readout Effort (min)','Readout Duration (min)','Follow Up Effort (min)']]

In [ ]:
familiar_relation = ca3.copy()

In [ ]:
familiar_relation.insert(familiar_relation.shape[1], 'Error relation Account Stage, Status columns', None)
familiar_relation.insert(familiar_relation.shape[1], 'Error Reason Declined', None)
familiar_relation.insert(familiar_relation.shape[1], 'Pre-ReadOut Empty', None)
familiar_relation.insert(familiar_relation.shape[1], 'Empty values in Readout Duration and Follow Up Effort', None)

In [ ]:
for index, status in familiar_relation.iterrows():
    if pd.isnull(status['Account Stage']) == False and pd.isnull(status['Status']) == False:
        if status['Status'][0] != status['Account Stage'][0]:
            familiar_relation['Error relation Account Stage, Status columns'].loc[index] = 'Error'
        else:
            if status['Account Stage'][0] == 'E' and status['Status'][2]== 'D' and pd.isnull(status['Reason Declined']) == True:
                familiar_relation['Error Reason Declined'].loc[index] = 'Error'
        
    if type(status['Pre-Readout Effort (min)']) != int:
        if pd.isnull(status['Pre-Readout Effort (min)']) == True:
            if pd.isnull(status['Status']) == False:
                if status['Status'][2] != 'U':
                    familiar_relation['Pre-ReadOut Empty'].loc[index] = 'Error'

    if pd.isnull(status['Account Stage']) == False:
        if status['Account Stage'][0] == 'F' and (pd.isnull(status['Readout Duration (min)']) == True or pd.isnull(status['Follow Up Effort (min)']) == True):
            familiar_relation['Empty values in Readout Duration and Follow Up Effort'].loc[index] = 'Error'   

In [ ]:
familiar_relation.drop(columns=familiar_relation.columns[1:7], inplace=True)

### Compare column AI-AL


In [ ]:
cov = ca[['Request ID','Follow Up 1','Follow Up 2','Follow Up 3','Closed Date']]

In [ ]:
equality_relation2 = cov.copy()

In [ ]:
equality_relation2['Follow Up 1'] = format_date_column(equality_relation2['Follow Up 1'])
equality_relation2['Follow Up 2'] = format_date_column(equality_relation2['Follow Up 2'])
equality_relation2['Follow Up 3'] = format_date_column(equality_relation2['Follow Up 3'])
equality_relation2['Closed Date'] = format_date_column(equality_relation2['Closed Date'])

In [ ]:
equality_relation2['Follow Up 1_greater_Follow Up 2'] = None
equality_relation2['Follow Up 2_greater_Follow Up 3'] = None
equality_relation2['Follow Up 3_greater_Closed Date'] = None

In [ ]:
for i, value in equality_relation2.iterrows():
    if value['Follow Up 1'] > value['Follow Up 2']:
        if value['Follow Up 1'] == Timestamp('2030-2-2 00:00:00') or value['Follow Up 1'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation2['Follow Up 1_greater_Follow Up 2'].loc[i] = None
        else:
            equality_relation2['Follow Up 1_greater_Follow Up 2'].loc[i] = 'X'

    if value['Follow Up 2'] > value['Follow Up 3']:
        if value['Follow Up 2'] == Timestamp('2030-2-2 00:00:00') or value['Follow Up 2'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation2['Follow Up 2_greater_Follow Up 3'].loc[i] = None
        else:
            equality_relation2['Follow Up 2_greater_Follow Up 3'].loc[i] = 'X'  

    if value['Follow Up 3'] > value['Closed Date']:
        if value['Follow Up 3'] == Timestamp('2030-2-2 00:00:00') or value['Follow Up 3'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation2['Follow Up 3_greater_Closed Date'].loc[i] = None  
        else:
            equality_relation2['Follow Up 3_greater_Closed Date'].loc[i] = 'X'

In [ ]:
for i, val in equality_relation2.iterrows():

    if val['Follow Up 1'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation2['Follow Up 1'].loc[i] = 'Format Error'
    else:
        equality_relation2['Follow Up 1'].loc[i] = None

    if val['Follow Up 2'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation2['Follow Up 2'].loc[i] = 'Format Error'
    else:
        equality_relation2['Follow Up 2'].loc[i] = None
    
    if val['Follow Up 3'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation2['Follow Up 3'].loc[i] = 'Format Error'
    else:
        equality_relation2['Follow Up 3'].loc[i] = None

    if val['Closed Date'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation2['Closed Date'].loc[i] = 'Format Error'
    else:
        equality_relation2['Closed Date'].loc[i] = None
    

### Merging tables

In [ ]:
new_table = equality_relation.merge(familiar_relation, how='left', left_on='Request ID', right_on='Request ID').merge(    
            Empty_Errors, how='left', left_on='Request ID', right_on='Request ID').merge(
            equality_relation2, how='left', left_on='Request ID', right_on='Request ID')
new_table.dropna(axis=0, how='all', subset=new_table.columns[2:], inplace=True)      
new_table.to_excel('error_report_%s.xlsx' % today)

excel_formating.format_excel()

In [ ]:
mail_table = new_table.drop(columns=new_table.columns[2:])
mail_table = mail_table['Coverage Advisor'].to_frame().drop_duplicates().reset_index(drop=True)
mail_table = mail_table.merge(mail, how='left', left_on='Coverage Advisor', right_on='Coverage Advisor')
mail_table.dropna(axis=0, subset=['Email'], inplace=True)

### Send Mail

In [ ]:
#outlook server connection
outlServer = smtplib.SMTP(host='smtp-mail.outlook.com', port=587)

#Initialize encryption
outlServer.starttls()

#Login in the outlook account using the define variables username and password
outlServer.login(myMail, password)

In [ ]:
#Definition of the function to send emails
def send_msg(To, From, Subject, Message, s_attach):
    msg = MIMEMultipart()
    msg['From'] = From
    msg['To'] = To
    msg['Subject'] = Subject
    msg_content = MIMEText(Message, 'html', 'utf-8')
    msg.attach(msg_content)
       
    if s_attach == True:
        with open('error_report_%s.xlsx' % today, 'rb') as e:
            exc = MIMEBase('application', 'octet-stream', filename='error_report_%s.xlsx' % today)
            exc.add_header('content-Disposition', 'attachment', filename='error_report_%s.xlsx' % today)
            exc.set_payload(e.read())
            encoders.encode_base64(exc)
            msg.attach(exc)
    
    outlServer.sendmail(From, To, msg.as_string())

In [ ]:
for i, sheet in mail_table.iterrows():
    if pd.isnull(sheet['Email']) == False:
        send_msg(sheet['Email'], myMail, subjectEmail , msg1, True)
        print(sheet['Email'])

In [ ]:
outlServer.quit()